In [3]:
import json
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import os
import requests

# Download All Tournament Games

In [181]:
loaded_tournaments = {}
for file in os.listdir("../Data/Games"):
    tournament = "_".join(file.split("_")[0].split("-")[:-1])
    if tournament not in loaded_tournaments:
        loaded_tournaments[tournament] = 0
    loaded_tournaments[tournament] += 1

In [4]:
def downloadTournament(link, tournament):
    skipped = 0
    print("\033[32mDownloading {} games\033[0m".format(tournament))
    
    page = 1
    all_games = set()

    while True:
        if page > 1000:
            break
        paginated_link = f"{link}?&players={page}"
        ht = requests.get(paginated_link)
        soup = BeautifulSoup(ht.content, 'html.parser')
        hrefs = soup.find_all('a', href=True)
        games = [x["href"] for x in hrefs if "game" in x['href']]
        games = set(games)
        if len(games) == 0:
            break
        
        # Check if all games are already in all_games
        if all([x in all_games for x in games]):
            break

        # Add to all_games
        all_games = all_games.union(games)
        page += 1
        print(f"\033[32mPage {page} ({len(games)}) loaded.\033[0m", end = "")

    if len(all_games) == 0:
        print(f"\033[31mNo games found for {tournament}\033[0m")
        return

    for game in tqdm(all_games, desc=f"Games in {tournament}", leave=False):
        id = game.split("/")[-1]
        if os.path.exists(f"../Data/Games/{tournament}_{id}.json"):
            skipped += 1
            continue

        first_game = "https://www.chess.com/callback/live/game/" + id
        ht = requests.get(first_game)
        game_data = ht.json()
        with open(f"../Data/Games/{tournament}_{id}.json", "w") as f:
            json.dump(game_data, f)

    print("\033[31mDownloaded {} games and skipped {}\033[0m".format(len(all_games), skipped))


In [5]:
tournaments = []

tournaments_link = f"https://www.chess.com/tournament/live/titled-tuesdays?&page="
for i in range(1,7):
    ht = requests.get(tournaments_link + str(i))
    soup = BeautifulSoup(ht.content, 'html.parser')
    hrefs = soup.find_all('a', href=True)
    links = [x["href"] for x in hrefs if "tournament" in x['href'] and "returnUrl" not in x["href"]]

    for url in set(links):
        if 'chess.com/tournament/live/' in url and 'titled-tuesday' in url:
            parts = url.split('/')
            name = parts[-1]
            date = '-'.join(name.split('-')[-4:-1])
            tournaments.append({'url': url, 'name': name, 'date': date})

In [10]:
for tournament in tournaments:
    downloadTournament(tournament['url'], tournament['name'])

Page 2 (239) loaded.Page 3 (272) loaded.Page 4 (280) loaded.Page 5 (282) loaded.Page 6 (288) loaded.Page 7 (290) loaded.Page 8 (280) loaded.Page 9 (292) loaded.Page 10 (289) loaded.Page 11 (271) loaded.Page 12 (280) loaded.Page 13 (271) loaded.Page 14 (275) loaded.Page 15 (215) loaded.Page 16 (134) loaded.Page 17 (227) loaded.Page 18 (204) loaded.Page 19 (172) loaded.Page 20 (190) loaded.Page 21 (155) loaded.Page 22 (106) loaded.Page 23 (109) loaded.

Downloaded 2439 games and skipped 2439
Page 2 (240) loaded.Page 3 (271) loaded.Page 4 (273) loaded.Page 5 (281) loaded.Page 6 (277) loaded.Page 7 (284) loaded.Page 8 (273) loaded.Page 9 (266) loaded.Page 10 (227) loaded.Page 11 (159) loaded.Page 12 (210) loaded.Page 13 (183) loaded.Page 14 (174) loaded.Page 15 (120) loaded.Page 16 (96) loaded.

Downloaded 1625 games and skipped 1625
Page 2 (245) loaded.Page 3 (275) loaded.Page 4 (281) loaded.Page 5 (290) loaded.Page 6 (290) loaded.Page 7 (289) loaded.Page 8 (281) loaded.Page 9 (280) loaded.Page 10 (289) loaded.Page 11 (277) loaded.Page 12 (272) loaded.Page 13 (263) loaded.Page 14 (273) loaded.Page 15 (219) loaded.Page 16 (148) loaded.Page 17 (173) loaded.Page 18 (233) loaded.Page 19 (198) loaded.Page 20 (182) loaded.Page 21 (167) loaded.Page 22 (139) loaded.Page 23 (116) loaded.Page 24 (117) loaded.Page 25 (66) loaded.

Downloaded 2539 games and skipped 2539
Page 2 (245) loaded.Page 3 (271) loaded.Page 4 (276) loaded.Page 5 (284) loaded.Page 6 (292) loaded.Page 7 (291) loaded.Page 8 (286) loaded.Page 9 (281) loaded.Page 10 (284) loaded.Page 11 (278) loaded.Page 12 (283) loaded.Page 13 (290) loaded.Page 14 (252) loaded.Page 15 (231) loaded.Page 16 (190) loaded.Page 17 (110) loaded.Page 18 (225) loaded.Page 19 (210) loaded.Page 20 (197) loaded.Page 21 (162) loaded.Page 22 (150) loaded.Page 23 (102) loaded.Page 24 (105) loaded.

Downloaded 2528 games and skipped 2528
Page 2 (240) loaded.Page 3 (262) loaded.Page 4 (277) loaded.Page 5 (279) loaded.Page 6 (286) loaded.Page 7 (281) loaded.Page 8 (254) loaded.Page 9 (264) loaded.Page 10 (236) loaded.Page 11 (147) loaded.Page 12 (198) loaded.Page 13 (211) loaded.Page 14 (183) loaded.Page 15 (168) loaded.Page 16 (155) loaded.Page 17 (107) loaded.Page 18 (96) loaded.

...

Downloaded 1615 games and skipped 0
Page 2 (239) loaded.Page 3 (259) loaded.Page 4 (275) loaded.Page 5 (278) loaded.Page 6 (275) loaded.Page 7 (278) loaded.Page 8 (271) loaded.Page 9 (235) loaded.Page 10 (123) loaded.Page 11 (200) loaded.Page 12 (167) loaded.Page 13 (150) loaded.Page 14 (104) loaded.

Downloaded 1399 games and skipped 0
Page 2 (241) loaded.Page 3 (264) loaded.Page 4 (271) loaded.Page 5 (280) loaded.Page 6 (282) loaded.Page 7 (275) loaded.Page 8 (225) loaded.Page 9 (162) loaded.Page 10 (197) loaded.Page 11 (204) loaded.Page 12 (164) loaded.Page 13 (126) loaded.Page 14 (98) loaded.

Downloaded 1357 games and skipped 0
Page 2 (245) loaded.Page 3 (263) loaded.Page 4 (270) loaded.Page 5 (275) loaded.Page 6 (275) loaded.Page 7 (266) loaded.Page 8 (262) loaded.Page 9 (258) loaded.Page 10 (171) loaded.Page 11 (170) loaded.Page 12 (191) loaded.Page 13 (163) loaded.Page 14 (141) loaded.Page 15 (110) loaded.

Downloaded 1493 games and skipped 0
Page 2 (236) loaded.Page 3 (264) loaded.Page 4 (278) loaded.Page 5 (294) loaded.Page 6 (288) loaded.Page 7 (269) loaded.Page 8 (283) loaded.Page 9 (282) loaded.Page 10 (281) loaded.Page 11 (249) loaded.Page 12 (158) loaded.Page 13 (226) loaded.Page 14 (189) loaded.Page 15 (169) loaded.Page 16 (149) loaded.Page 17 (104) loaded.

Games in early-titled-tuesday-blitz-october-04-2022-3446088:  41%|████▏     | 744/1797 [04:00<04:36,  3.81it/s]